In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
from PIL import Image
import io
from pyspark.sql.functions import expr


In [2]:
spark = SparkSession.builder.appName("ImageDatabase").getOrCreate()

In [3]:
# load from folder
image_df = spark.read.format("image").option("dropInvalid", True).load("hdfs://localhost:9000/user1/weather")


In [4]:
# create database with images
image_df.createOrReplaceTempView("image_database")


In [5]:
# print schema
image_df.printSchema()


root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)



In [6]:
# list tables
table_list = spark.sql("SHOW TABLES").collect()
for table in table_list:
    print(table)


2023-09-23 19:23:09,447 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2023-09-23 19:23:09,450 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist
2023-09-23 19:23:12,837 WARN metastore.ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
2023-09-23 19:23:12,837 WARN metastore.ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore hduser@127.0.1.1
2023-09-23 19:23:13,285 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


Row(namespace='', tableName='image_database', isTemporary=True)


In [7]:
# from database to dataframe
image_df = spark.sql("SELECT * FROM image_database")


In [8]:
def resize(image_bytes):
    # load image
    image = Image.open(io.BytesIO(image_bytes))
    
    # rsize
    resized_image = image.resize((224, 224))
    
    # Convert in byte
    output_buffer = io.BytesIO()
    resized_image.save(output_buffer, format="JPEG")
    resized_image_bytes = output_buffer.getvalue()
    
    return resized_image_bytes

# udf function
udf = udf(resize, StringType())

# Resize in dataframe
df2 = image_df.withColumn("resized_image", udf(col("image")))

In [9]:
spark = SparkSession.builder.appName("ImageDatabase").config("spark.driver.memory", "8g").getOrCreate()


In [10]:
# retrieve data

first_row = df2.first()
image_bytes = first_row.resized_image
image = Image.open(io.BytesIO(image_bytes))
image.show()



2023-09-23 19:23:49,455 ERROR util.Utils: Uncaught exception in thread stdout writer for python3
java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOfRange(Arrays.java:3664)
	at java.lang.String.<init>(String.java:207)
	at java.lang.StringBuilder.toString(StringBuilder.java:412)
	at net.razorvine.pickle.PickleUtils.rawStringFromBytes(PickleUtils.java:294)
	at net.razorvine.pickle.Pickler.put_arrayOfPrimitives(Pickler.java:622)
	at net.razorvine.pickle.Pickler.dispatch(Pickler.java:260)
	at net.razorvine.pickle.Pickler.save(Pickler.java:193)
	at org.apache.spark.sql.execution.python.EvaluatePython$RowPickler.pickle(EvaluatePython.scala:271)
	at net.razorvine.pickle.Pickler.dispatch(Pickler.java:305)
	at net.razorvine.pickle.Pickler.save(Pickler.java:193)
	at org.apache.spark.sql.execution.python.EvaluatePython$RowPickler.pickle(EvaluatePython.scala:271)
	at net.razorvine.pickle.Pickler.dispatch(Pickler.java:305)
	at net.razorvine.pickle.Pickler.save(Pickler.java:193)
	

KeyboardInterrupt: 